In [1]:
import astropy.time
from lsst.daf.butler import Butler, Timespan

In [2]:
from lsst.daf.butler import CollectionType

In [3]:
butler = Butler('/repo/embargo')
registry = butler.registry

In [4]:
data_path = '/home/j/jarugula/DM-37690/transfer_embargo/tests/data/'
test_from_butler = Butler(data_path+'test_from', writeable=True)
test_from_butler_registry = test_from_butler.registry

In [5]:
datasetType_calexp = 'calexp'
datasetType_raw = 'raw'
collections_calexp = 'LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616'
collections_raw = 'LATISS/raw/all'
dataId = {'instrument':'LATISS'}

In [8]:
now = astropy.time.Time.now().tai
# embargo_hours = 80.0
embargo_hours = 200.0
embargo_period = astropy.time.TimeDelta(
        embargo_hours * 3600.0, format="sec"
    )
timespan_embargo = Timespan(now - embargo_period, None)

In [9]:
datarefs_calexp = registry.queryDimensionRecords(
            "exposure",
            dataId=dataId,
            datasets=datasetType_calexp,
            collections=collections_calexp,
            where="NOT exposure.timespan OVERLAPS timespan_embargo",
            bind={"timespan_embargo": timespan_embargo},
        )

In [10]:
exp_calexp = []
for i, ref in enumerate(datarefs_calexp):
    exp_calexp.append(ref.dataId['exposure'])

In [14]:
datarefs_raw = registry.queryDimensionRecords(
            "exposure",
            dataId=dataId,
            datasets=datasetType_raw,
            collections=collections_raw,
            where="NOT exposure.timespan OVERLAPS timespan_embargo",
            bind={"timespan_embargo": timespan_embargo},
        )

In [15]:
exp_raw = []
for i, ref in enumerate(datarefs_raw):
    exp_raw.append(ref.dataId['exposure'])

In [16]:
# exposure_ids = [2022111000436, 2022110900673, 2022110800580]
exposure_ids = [2022110800230, 2022110800235, 2022110800238]

In [17]:
datasetRefs_calexp = registry.queryDatasets(datasetType_calexp, dataId=dataId, collections=collections_calexp,
                                     where="exposure.id IN (exposure_ids)",
                                     bind={"exposure_ids": exposure_ids}).expanded()
datasetRefs_raw = registry.queryDatasets(datasetType_raw, dataId=dataId, collections=collections_raw,
                                     where="exposure.id IN (exposure_ids)",
                                     bind={"exposure_ids": exposure_ids}).expanded()

In [18]:
for i,ref in enumerate(datasetRefs_calexp):
    print(ref.dataId)

{instrument: 'LATISS', detector: 0, visit: 2022110800230, ...}
{instrument: 'LATISS', detector: 0, visit: 2022110800235, ...}
{instrument: 'LATISS', detector: 0, visit: 2022110800238, ...}


In [19]:
for i,ref in enumerate(datasetRefs_calexp):
    print(ref)

calexp@{instrument: 'LATISS', detector: 0, visit: 2022110800230, ...} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z id=d15ad796-1855-434d-b06e-ac3fd6a1ec19)
calexp@{instrument: 'LATISS', detector: 0, visit: 2022110800235, ...} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z id=1f6ac49d-8aa3-4b56-a65c-7566bdb5c9be)
calexp@{instrument: 'LATISS', detector: 0, visit: 2022110800238, ...} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z id=d7a7dad4-ff96-4d4b-9960-87285483a44e)


In [20]:
for i,ref in enumerate(datasetRefs_raw):
    print(ref)

raw@{instrument: 'LATISS', detector: 0, exposure: 2022110800230, ...} [sc=Exposure] (run=LATISS/raw/all id=17be4a18-8fd2-5b57-a016-d113be349482)
raw@{instrument: 'LATISS', detector: 0, exposure: 2022110800235, ...} [sc=Exposure] (run=LATISS/raw/all id=fe75549a-b0f7-5000-a18b-63093c341ac1)
raw@{instrument: 'LATISS', detector: 0, exposure: 2022110800238, ...} [sc=Exposure] (run=LATISS/raw/all id=80457bf6-4c25-5fb9-a763-6c94b258bb9e)


In [21]:
test_from_butler.transfer_from(butler, source_refs=datasetRefs_calexp, transfer='copy',
                               skip_missing=True, register_dataset_types=True,
                               transfer_dimensions=True)

[DatasetRef(DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF), {instrument: 'LATISS', detector: 0, visit: 2022110800235, ...}, run='LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z', id=1f6ac49d-8aa3-4b56-a65c-7566bdb5c9be),
 DatasetRef(DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF), {instrument: 'LATISS', detector: 0, visit: 2022110800230, ...}, run='LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z', id=d15ad796-1855-434d-b06e-ac3fd6a1ec19),
 DatasetRef(DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF), {instrument: 'LATISS', detector: 0, visit: 2022110800238, ...}, run='LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z', id=d7a7dad4-ff96-4d4b-9960-87285483a44e)]

In [22]:
test_from_butler.transfer_from(butler, source_refs=datasetRefs_raw, transfer='copy',
                               skip_missing=True, register_dataset_types=True,
                               transfer_dimensions=True)

[DatasetRef(DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure), {instrument: 'LATISS', detector: 0, exposure: 2022110800230, ...}, run='LATISS/raw/all', id=17be4a18-8fd2-5b57-a016-d113be349482),
 DatasetRef(DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure), {instrument: 'LATISS', detector: 0, exposure: 2022110800238, ...}, run='LATISS/raw/all', id=80457bf6-4c25-5fb9-a763-6c94b258bb9e),
 DatasetRef(DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure), {instrument: 'LATISS', detector: 0, exposure: 2022110800235, ...}, run='LATISS/raw/all', id=fe75549a-b0f7-5000-a18b-63093c341ac1)]

### check if we can query based on exposure

In [25]:
# testrefs = test_from_butler_registry.queryDimensionRecords("exposure",  dataId=dataId, datasets='raw', collections=...)
testrefs = test_from_butler_registry.queryDimensionRecords("exposure",  dataId=dataId, datasets='calexp', collections=...)
for i, ref in enumerate(testrefs):
    print(ref)

In [26]:
testrefs = test_from_butler_registry.queryDimensionRecords("visit",  dataId=dataId, datasets='calexp', collections=...)
for i, ref in enumerate(testrefs):
    print(ref)

visit:
  instrument: 'LATISS'
  id: 2022110800230
  physical_filter: 'SDSSi_65mm~empty'
  name: 'AT_O_20221108_000230'
  day_obs: 20221108
  seq_num: 230
  exposure_time: 30.0
  target_name: 'HS2027+0651_047'
  observation_reason: 'object'
  science_program: 'AUXTEL_DRP_IMAGING'
  azimuth: 303.7437208776828
  zenith_angle: 56.618149818561506
  region: ConvexPolygon([UnitVector3d(0.6039059705203078, -0.7873621798438397, 0.12392891721257829), UnitVector3d(0.6051505939492327, -0.7866507644979353, 0.12236557259991532), UnitVector3d(0.6062740338792239, -0.7855679883134805, 0.12375270332774606), UnitVector3d(0.6050294103844203, -0.7862794037198054, 0.1253160478785859)])
  timespan: Timespan(begin=astropy.time.Time('2022-11-09 01:00:04.354016', scale='tai', format='iso'), end=astropy.time.Time('2022-11-09 01:00:34.587000', scale='tai', format='iso'))
visit:
  instrument: 'LATISS'
  id: 2022110800235
  physical_filter: 'SDSSi_65mm~empty'
  name: 'AT_O_20221108_000235'
  day_obs: 20221108
  seq

## Set Collection chain

In [13]:
#get chained collection
collection_chain = registry.getCollectionChain(collections)

In [14]:
collection_chain

('LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z',
 'LATISS/raw/all',
 'LATISS/calib/DM-36719/bias.20221107',
 'LATISS/calib/DM-36719/dark.20221107',
 'LATISS/calib/DM-36719/flat-SDSSi.20221107',
 'LATISS/calib/DM-36719/flat-SDSSr.20221107',
 'LATISS/calib/DM-36719/flat-SDSSg.20221107',
 'LATISS/calib/DM-36484/bias.20221005a',
 'LATISS/calib/DM-36484/dark.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSg.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSr.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSi.20221006a',
 'u/czw/defects.20220608',
 'LATISS/calib/DM-33875',
 'u/czw/DM-28920/calib/bias.20210720',
 'u/czw/DM-28920/calib/dark.20210720a',
 'u/calib/DM-32209-20211013a-g',
 'u/calib/DM-32209-20211013a-felh',
 'u/czw/DM-28920/calib/flat.20210720',
 'u/czw/DM-28920/calib/defect.20210720a',
 'LATISS/calib/DM-28636',
 'LATISS/calib/DM-28636/unbounded',
 'refcats/DM-28636',
 'refcats/DM-33444',
 'skymaps')

In [53]:
for c in collection_chain:
    print(c)

LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z
LATISS/raw/all
LATISS/calib/DM-36719/bias.20221107
LATISS/calib/DM-36719/dark.20221107
LATISS/calib/DM-36719/flat-SDSSi.20221107
LATISS/calib/DM-36719/flat-SDSSr.20221107
LATISS/calib/DM-36719/flat-SDSSg.20221107
LATISS/calib/DM-36484/bias.20221005a
LATISS/calib/DM-36484/dark.20221006a
LATISS/calib/DM-36484/flat-SDSSg.20221006a
LATISS/calib/DM-36484/flat-SDSSr.20221006a
LATISS/calib/DM-36484/flat-SDSSi.20221006a
u/czw/defects.20220608
LATISS/calib/DM-33875
u/czw/DM-28920/calib/bias.20210720
u/czw/DM-28920/calib/dark.20210720a
u/calib/DM-32209-20211013a-g
u/calib/DM-32209-20211013a-felh
u/czw/DM-28920/calib/flat.20210720
u/czw/DM-28920/calib/defect.20210720a
LATISS/calib/DM-28636
LATISS/calib/DM-28636/unbounded
refcats/DM-28636
refcats/DM-33444
skymaps


In [21]:
registry.getCollectionSummary(collections)

CollectionSummary(dataset_types=NamedValueSet({DatasetType('transformSourceTable_metadata', {band, instrument, detector, physical_filter, visit}, TaskMetadata), DatasetType('consolidateSourceTable_metadata', {band, instrument, physical_filter, visit}, TaskMetadata), DatasetType('isr_log', {band, instrument, detector, physical_filter, exposure}, ButlerLogRecords), DatasetType('isr_metadata', {band, instrument, detector, physical_filter, exposure}, TaskMetadata), DatasetType('sourceTable_visit', {band, instrument, physical_filter, visit}, DataFrame), DatasetType('transformSourceTable_log', {band, instrument, detector, physical_filter, visit}, ButlerLogRecords), DatasetType('characterizeImage_log', {band, instrument, detector, physical_filter, visit}, ButlerLogRecords), DatasetType('calibrate_log', {band, instrument, detector, physical_filter, visit}, ButlerLogRecords), DatasetType('icExp', {band, instrument, detector, physical_filter, visit}, ExposureF), DatasetType('src', {band, instrum

In [24]:
test_from_butler_registry.getCollectionSummary('LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z')

CollectionSummary(dataset_types=NamedValueSet({DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF)}), governors={'instrument': {'LATISS'}})

In [54]:
# register all the collections from the chain
for c in collection_chain:
    test_from_butler_registry.registerCollection(c, CollectionType(3))

In [47]:
# test_from_butler_registry.removeCollection('test_chained_collection')

In [55]:
for c in sorted(test_from_butler_registry.queryCollections(datasetType='calexp')):
    print(c)

LATISS/calib/DM-28636
LATISS/calib/DM-28636/unbounded
LATISS/calib/DM-33875
LATISS/calib/DM-36484/bias.20221005a
LATISS/calib/DM-36484/dark.20221006a
LATISS/calib/DM-36484/flat-SDSSg.20221006a
LATISS/calib/DM-36484/flat-SDSSi.20221006a
LATISS/calib/DM-36484/flat-SDSSr.20221006a
LATISS/calib/DM-36719/bias.20221107
LATISS/calib/DM-36719/dark.20221107
LATISS/calib/DM-36719/flat-SDSSg.20221107
LATISS/calib/DM-36719/flat-SDSSi.20221107
LATISS/calib/DM-36719/flat-SDSSr.20221107
LATISS/raw/all
LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616
LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z
refcats/DM-28636
refcats/DM-33444
skymaps
u/calib/DM-32209-20211013a-felh
u/calib/DM-32209-20211013a-g
u/czw/DM-28920/calib/bias.20210720
u/czw/DM-28920/calib/dark.20210720a
u/czw/DM-28920/calib/defect.20210720a
u/czw/DM-28920/calib/flat.20210720
u/czw/defects.20220608


In [56]:
test_from_butler_registry.getCollectionChain(collections)

()

In [57]:
# add chained collection to the destination butler
test_from_butler_registry.setCollectionChain(parent=collections, children=collection_chain)

In [66]:
test_from_butler_registry.getCollectionChain(collections)

('LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z',
 'LATISS/raw/all',
 'LATISS/calib/DM-36719/bias.20221107',
 'LATISS/calib/DM-36719/dark.20221107',
 'LATISS/calib/DM-36719/flat-SDSSi.20221107',
 'LATISS/calib/DM-36719/flat-SDSSr.20221107',
 'LATISS/calib/DM-36719/flat-SDSSg.20221107',
 'LATISS/calib/DM-36484/bias.20221005a',
 'LATISS/calib/DM-36484/dark.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSg.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSr.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSi.20221006a',
 'u/czw/defects.20220608',
 'LATISS/calib/DM-33875',
 'u/czw/DM-28920/calib/bias.20210720',
 'u/czw/DM-28920/calib/dark.20210720a',
 'u/calib/DM-32209-20211013a-g',
 'u/calib/DM-32209-20211013a-felh',
 'u/czw/DM-28920/calib/flat.20210720',
 'u/czw/DM-28920/calib/defect.20210720a',
 'LATISS/calib/DM-28636',
 'LATISS/calib/DM-28636/unbounded',
 'refcats/DM-28636',
 'refcats/DM-33444',
 'skymaps')

In [67]:
test_from_butler_registry.getCollectionSummary(collections)

CollectionSummary(dataset_types=NamedValueSet({DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF), DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure)}), governors={'instrument': {'LATISS'}})

In [58]:
testrefs = test_from_butler_registry.queryDatasets(datasetType, dataId=dataId, collections=collections)
for i, ref in enumerate(testrefs):
    print(ref.dataId.full)

{band: 'i', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSi_65mm~empty', visit: 2022110800580}
{band: 'g', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSg_65mm~empty', visit: 2022110900673}
{band: 'i', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSi_65mm~empty', visit: 2022111000436}


In [59]:
testrefs = test_from_butler_registry.queryDatasets(datasetType, dataId=dataId, collections=...)
for i, ref in enumerate(testrefs):
    print(ref.dataId.full)

{band: 'i', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSi_65mm~empty', visit: 2022110800580}
{band: 'g', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSg_65mm~empty', visit: 2022110900673}
{band: 'i', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSi_65mm~empty', visit: 2022111000436}


In [64]:
# testrefs = test_from_butler_registry.queryDimensionRecords("exposure",  dataId=dataId, datasets=datasetType,collections=...)
testrefs = test_from_butler_registry.queryDimensionRecords("visit",  dataId=dataId, datasets=datasetType,collections=collections)
for i, ref in enumerate(testrefs):
    print(ref)

visit:
  instrument: 'LATISS'
  id: 2022110800580
  physical_filter: 'SDSSi_65mm~empty'
  name: 'AT_O_20221108_000580'
  day_obs: 20221108
  seq_num: 580
  exposure_time: 30.0
  target_name: 'SPT-CL-J0559-5249_063'
  observation_reason: 'object'
  science_program: 'AUXTEL_DRP_IMAGING'
  azimuth: 139.27777238604904
  zenith_angle: 36.25017562667199
  region: ConvexPolygon([UnitVector3d(-0.0014031665810288064, 0.6073223384722833, -0.7944542833392637), UnitVector3d(-0.0035242917214096405, 0.6073137483105336, -0.7944542721144321), UnitVector3d(-0.003530996614655318, 0.6089710242215567, -0.7931846088531064), UnitVector3d(-0.0014098714605740878, 0.608979614297738, -0.7931846201435344)])
  timespan: Timespan(begin=astropy.time.Time('2022-11-09 04:54:12.359010', scale='tai', format='iso'), end=astropy.time.Time('2022-11-09 04:54:42.592000', scale='tai', format='iso'))
visit:
  instrument: 'LATISS'
  id: 2022110900673
  physical_filter: 'SDSSg_65mm~empty'
  name: 'AT_O_20221109_000673'
  day_ob

In [65]:
testrefs = test_from_butler_registry.queryDimensionRecords("exposure",  dataId=dataId, datasets=datasetType,collections=collections)
for i, ref in enumerate(testrefs):
    print(ref)

In [18]:
outside_embargo = [
            dt.id
            for dt in test_from_butler_registry.queryDimensionRecords(
                "exposure",
                dataId=dataId,
                datasets=datasetType,
                # collections=collections_exposure,
                collections=...,)]

In [19]:
outside_embargo

[]

In [20]:
outside_embargo = [
            dt.id
            for dt in test_from_butler_registry.queryDimensionRecords(
                "exposure",
                dataId=dataId,
                datasets=datasetType,
                # collections=collections_exposure,
                collections=...,
                where="NOT exposure.timespan OVERLAPS\
                                                        timespan_embargo",
                bind={"timespan_embargo": timespan_embargo},
            )
        ]

In [21]:
outside_embargo

[]